In [2]:
import gymnasium as gym
import gymnasium_robotics
import numpy as np
from stable_baselines3 import SAC, HerReplayBuffer

In [ ]:
STAGES = {
    '1' : 'HOVER', 
    '2' : 'GRASP', 
    '3' : 'PLACE', 
}

In [69]:
env = gym.make('FetchPickAndPlace-v4', render_mode='human')
env.reset()

({'observation': array([ 1.34193475e+00,  7.49101049e-01,  5.34725189e-01,  1.37882812e+00,
          6.48572855e-01,  4.24892225e-01,  3.68933623e-02, -1.00528194e-01,
         -1.09832964e-01,  3.83587563e-06,  6.44539257e-08, -8.04518216e-19,
          1.42324495e-16,  1.16233816e-16,  5.26138825e-06,  7.50031136e-08,
         -2.45923720e-05, -1.87673947e-18, -2.11089242e-16,  1.88168714e-18,
         -5.26138825e-06, -7.50031136e-08,  2.46214240e-05, -9.88156877e-07,
          7.65319420e-08]),
  'achieved_goal': array([1.37882812, 0.64857285, 0.42489222]),
  'desired_goal': array([1.37918241, 0.64031931, 0.42489222])},
 {})

In [70]:
for name in dir(env):
    print(name)



__annotations__
__class__
__class_getitem__
__delattr__
__dict__
__dir__
__doc__
__enter__
__eq__
__exit__
__format__
__ge__
__getattribute__
__getstate__
__gt__
__hash__
__init__
__init_subclass__
__le__
__lt__
__module__
__ne__
__new__
__orig_bases__
__parameters__
__reduce__
__reduce_ex__
__repr__
__setattr__
__sizeof__
__str__
__subclasshook__
__weakref__
_action_space
_cached_spec
_elapsed_steps
_max_episode_steps
_metadata
_np_random
_np_random_seed
_observation_space
_saved_kwargs
action_space
class_name
close
env
get_wrapper_attr
has_wrapper_attr
metadata
np_random
np_random_seed
observation_space
render
render_mode
reset
set_wrapper_attr
spec
step
unwrapped
wrapper_spec


In [71]:

# Get just the methods
print("\nMethods only:")
for name in dir(env):
    attr = getattr(env, name)
    if callable(attr):
        print(name)


Methods only:
__class__
__class_getitem__
__delattr__
__dir__
__enter__
__eq__
__exit__
__format__
__ge__
__getattribute__
__getstate__
__gt__
__hash__
__init__
__init_subclass__
__le__
__lt__
__ne__
__new__
__reduce__
__reduce_ex__
__repr__
__setattr__
__sizeof__
__str__
__subclasshook__


AttributeError: Can't access `_np_random` of a wrapper, use `.unwrapped._np_random` or `.np_random`.

### finding the range of goals

In [78]:
goal_x, goal_y, goal_z = [], [], []

In [76]:
# env.reset()


In [82]:
for i in range(10000):
    obs, _ = env.reset()
    goal_x.append(obs['desired_goal'][0])
    goal_y.append(obs['desired_goal'][1])
    goal_z.append(obs['desired_goal'][2])

In [84]:
env.close()

In [83]:
min(goal_x), min(goal_y), min(goal_z), max(goal_x), max(goal_y), max(goal_z)

(np.float64(1.1919517529646197),
 np.float64(0.5991066228570885),
 np.float64(0.4248922232488584),
 np.float64(1.491934091203951),
 np.float64(0.8990975732816955),
 np.float64(0.8748198015890661))

from code:
object spawn center ≈ [1.25, 0.53, 0.4]
obj_range=0.15
target_range=0.15

therefore, 
x ∈ [1.25 - 0.15, 1.25 + 0.15] =>  [1.10, 1.40]
y ∈ [0.53 - 0.15, 0.53 + 0.15] =>  [0.38, 0.68]
0.4 (ground clearance) -> 0.4 + random(0, 0.4) = 0.4 -> 0.8


x ∈ [1.10, 1.40]
y ∈ [0.38, 0.68]
z ≈ 0.42 (on the table)

final:
x in [1.20, 1.48]
y in [0.62, 0.88]
z in [0.425, 0.875]

In [68]:
from gymnasium_robotics.envs.fetch.pick_and_place import MujocoFetchPickAndPlaceEnv
import inspect

print(MujocoFetchPickAndPlaceEnv.__mro__)  # shows the parent classes

# import gymnasium_robotics.envs.fetch.fetch_env.MujocoFetchEnv as base_fetch
# print(inspect.getsource(base_fetch.BaseFetchEnv.reset))

(<class 'gymnasium_robotics.envs.fetch.pick_and_place.MujocoFetchPickAndPlaceEnv'>, <class 'gymnasium_robotics.envs.fetch.fetch_env.MujocoFetchEnv'>, <class 'gymnasium_robotics.envs.fetch.fetch_env.get_base_fetch_env.<locals>.BaseFetchEnv'>, <class 'gymnasium_robotics.envs.robot_env.MujocoRobotEnv'>, <class 'gymnasium_robotics.envs.robot_env.BaseRobotEnv'>, <class 'gymnasium_robotics.core.GoalEnv'>, <class 'gymnasium.core.Env'>, <class 'typing.Generic'>, <class 'gymnasium.utils.ezpickle.EzPickle'>, <class 'object'>)


In [5]:
class CustomObs(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)
        
    def set_goal
        
    def reset(self, x, y, z):
        

AttributeError: 'TimeLimit' object has no attribute 'observation'

In [6]:
def set_custom_goal(env, obs, target_pos):
    """
    Updates the observation for the agent AND moves the visual red dot.
    
    Args:
        env: The Gymnasium environment
        obs: The current observation dictionary
        target_pos: [x, y, z] numpy array of the new target
        
    Returns:
        The modified observation dictionary
    """
    # 1. Hack the Agent's Input
    obs['desired_goal'] = target_pos.copy()
    
    # 2. Hack the Visuals (The Red Dot)
    try:
        # Note: Different versions of Gym/Mujoco access data differently.
        # This works for the standard Gymnasium-Robotics + Mujoco setup.
        site_id = env.unwrapped.model.site("target0").id
        env.unwrapped.model.site_pos[site_id] = target_pos
        env.unwrapped.data.forward()
    except:
        # If visuals fail (e.g. headless mode), just ignore it. 
        # The robot will still work.
        pass

    return obs



In [13]:
obs, info = env.reset()


In [14]:
target_from_slider = np.array([1.3, 0.6, 0.25])
obs = set_custom_goal(env, obs, target_from_slider)

In [6]:
# env = gym.make("FetchReach-v4")
obs, info = env.reset()

env.goal = np.array([1.25, 0.75, 0.5])
# Also update achieved + desired dict for agent
obs["desired_goal"] = env.goal.copy()

In [7]:
env.step(env.action_space.sample())

({'observation': array([ 1.32490612e+00,  7.52627706e-01,  5.64787106e-01,  1.20922710e+00,
          8.91670395e-01,  4.24892240e-01, -1.15679020e-01,  1.39042689e-01,
         -1.39894866e-01,  1.05613321e-05,  1.04728067e-04, -1.08868783e-18,
          3.34089087e-17,  1.19630606e-16,  1.50461182e-02, -2.96663660e-03,
         -2.58884151e-02, -1.89976501e-18, -5.50611670e-17, -4.93610409e-19,
         -1.50461182e-02,  2.96663660e-03,  2.58884222e-02,  5.28888059e-05,
          5.01231694e-05]),
  'achieved_goal': array([1.2092271 , 0.89167039, 0.42489224]),
  'desired_goal': array([1.25786782, 0.8720507 , 0.42489222])},
 np.float32(-1.0),
 False,
 False,
 {'is_success': np.float32(0.0)})

#### by modifying subclass

In [65]:
from gymnasium_robotics.envs.fetch.pick_and_place import MujocoFetchPickAndPlaceEnv

class FetchPickAndPlaceCustomGoal(MujocoFetchPickAndPlaceEnv):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.current_goal = None

    def set_goal(self, goal: np.ndarray):
        self.current_goal = goal.copy()

    def _sample_goal(self):
        assert self.current_goal is not None, "Call set_goal before reset"
        return self.current_goal.copy()

In [66]:
env = FetchPickAndPlaceCustomGoal(render_mode='human')

# you do some stuff here: ask LLM, look at an image, etc.
# you only know the point at runtime
goal1 = np.array([1.25, 0.75, 0.50])

env.set_goal(goal1)          # set it right before reset
obs, info = env.reset()
print("Episode 1 goal:", obs["desired_goal"])


Episode 1 goal: [1.25 0.75 0.5 ]


In [33]:
goal1 = np.array([1.5, 0.75, 0.45])

env.set_goal(goal1)          # set it right before reset
obs, info = env.reset()
print("Episode 1 goal:", obs["desired_goal"])


Episode 1 goal: [1.5  0.75 0.45]


In [51]:
env.step(env.action_space.sample())     

({'observation': array([ 1.35176167e+00,  7.63733241e-01,  5.87246855e-01,  1.23791458e+00,
          7.98351906e-01,  4.24892243e-01, -1.13847084e-01,  3.46186648e-02,
         -1.62354611e-01,  4.00974884e-02,  4.17101562e-02,  1.89022274e-17,
          1.88849544e-17,  1.19287698e-16, -6.43170103e-04, -2.07898149e-02,
         -1.94346395e-02, -1.20629910e-17, -2.61025536e-17, -3.39347966e-18,
          6.43170103e-04,  2.07898149e-02,  1.94346412e-02,  6.98444984e-02,
          7.09726124e-02]),
  'achieved_goal': array([1.23791458, 0.79835191, 0.42489224]),
  'desired_goal': array([1.47198361, 0.68021574, 0.52509059])},
 np.float32(-1.0),
 False,
 False,
 {'is_success': np.float32(0.0)})

In [67]:
env.close()

#### by adding wrapper

In [ ]:
import gymnasium as gym
import numpy as np

class CustomGoalWrapper(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)
        self.current_goal = None

    def set_goal(self, goal):
        """Set goal for the *next* reset."""
        goal = np.asarray(goal, dtype=np.float32)
        self.current_goal = goal

    def _apply_goal(self, obs):
        """Update internal goal, desired_goal, and the red target site."""
        if self.current_goal is None:
            return obs

        g = self.current_goal.copy()

        # goal used by reward and HER
        self.env.goal = g

        # move the red target site in MuJoCo so the dot moves
        self.env.model.site_pos[self.env.target_site_id] = g

        # update observation so the agent sees the new goal
        obs["desired_goal"] = g

        return obs

    def reset(self, *, goal=None, **kwargs):
        """
        Optional argument:
            goal: np.array(3,)    desired target position for the object
        """
        if goal is not None:
            self.set_goal(goal)

        obs, info = self.env.reset(**kwargs)
        obs = self._apply_goal(obs)
        return obs, info

In [ ]:
class CustomGoalWrapper(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)
        self.current_goal = None
        
    def set_goal(self, goal):
        self.current_goal = goal
        
    def _apply_goal(self, obs):
        if self.current_goal is not None:
            g = self.current_goal.copy()
            
            self.env.goal = g
            self.env.model.site_pos[self.env.target_site_id] = g
            obs['desired_goal'] = g
            
        return obs

    def reset(self, goal, **kwargs):
        if self.goal is not None
            self.set_goal(goal)
        
        obs, info = self.reset(**kwargs):
        obs = self._apply_goal(obs)
        
        return obs, info

In [14]:
import os
import numpy as np
import gymnasium as gym
import gymnasium_robotics
from gymnasium_robotics.envs.fetch.fetch_env import MujocoFetchEnv

# Build the path to the multi-object XML inside gymnasium_robotics
GYMROB_ROOT = os.path.dirname(gymnasium_robotics.__file__)
MODEL_XML_PATH = os.path.join(
    GYMROB_ROOT,
    "envs",
    "assets",
    "fetch",
    "multi_pick_and_place.xml",
)


class MultiObjectFetchPickAndPlaceEnv(MujocoFetchEnv):
    def __init__(self, reward_type: str = "sparse", **kwargs):
        # You must set initial_qpos for every object's free joint
        initial_qpos = {
            "robot0:slide0": 0.405,
            "robot0:slide1": 0.48,
            "robot0:slide2": 0.0,
            # main object (object0) roughly like default
            "object0:joint": [1.25, 0.70, 0.42, 1.0, 0.0, 0.0, 0.0],
            # extra objects scattered on the table
            "object1:joint": [1.30, 0.80, 0.42, 1.0, 0.0, 0.0, 0.0],
            "object2:joint": [1.35, 0.60, 0.42, 1.0, 0.0, 0.0, 0.0],
            "object3:joint": [1.40, 0.75, 0.42, 1.0, 0.0, 0.0, 0.0],
            "object4:joint": [1.45, 0.65, 0.42, 1.0, 0.0, 0.0, 0.0],
        }

        super().__init__(
            model_path=MODEL_XML_PATH,
            has_object=True,
            block_gripper=False,
            n_substeps=20,
            gripper_extra_height=0.2,
            target_in_the_air=True,
            target_offset=0.0,
            obj_range=0.15,
            target_range=0.15,
            distance_threshold=0.05,
            initial_qpos=initial_qpos,
            reward_type=reward_type,
            **kwargs,
        )

In [18]:
# from multi_fetch_env import MultiObjectFetchPickAndPlaceEnv

env = MultiObjectFetchPickAndPlaceEnv(render_mode="human")
obs, info = env.reset()

In [21]:
env.close()

In [19]:
env.reset()

({'observation': array([ 1.34193475e+00,  7.49101049e-01,  5.34725189e-01,  1.24736527e+00,
          8.68241707e-01,  4.24892240e-01, -9.45694868e-02,  1.19140658e-01,
         -1.09832949e-01,  3.83587563e-06,  6.44539257e-08, -3.83126585e-18,
         -1.91079569e-16, -5.11864729e-14,  5.26138825e-06,  7.50031136e-08,
         -2.46153366e-05, -1.29834876e-17,  2.94396616e-16, -6.32178674e-19,
         -5.26138825e-06, -7.50031136e-08,  2.46214240e-05, -9.88156877e-07,
          7.65319420e-08]),
  'achieved_goal': array([1.24736527, 0.86824171, 0.42489224]),
  'desired_goal': array([1.43710006, 0.89503746, 0.42489224])},
 {})

In [20]:
env.reset()

({'observation': array([ 1.34193475e+00,  7.49101049e-01,  5.34725189e-01,  1.20239469e+00,
          8.03342837e-01,  4.24892240e-01, -1.39540063e-01,  5.42417889e-02,
         -1.09832949e-01,  3.83587563e-06,  6.44539257e-08, -3.83126585e-18,
         -1.91079569e-16, -5.11864729e-14,  5.26138825e-06,  7.50031136e-08,
         -2.46153366e-05, -1.29834876e-17,  2.94396616e-16, -6.32178674e-19,
         -5.26138825e-06, -7.50031136e-08,  2.46214240e-05, -9.88156877e-07,
          7.65319420e-08]),
  'achieved_goal': array([1.20239469, 0.80334284, 0.42489224]),
  'desired_goal': array([1.2633913 , 0.78164566, 0.51935622])},
 {})

In [12]:
env.reset()

({'observation': array([ 1.34193475e+00,  7.49101049e-01,  5.34725189e-01,  1.20576514e+00,
          6.03752801e-01,  4.24892240e-01, -1.36169612e-01, -1.45348247e-01,
         -1.09832949e-01,  3.83587563e-06,  6.44539257e-08, -3.83126585e-18,
         -1.91079569e-16, -5.11864729e-14,  5.26138825e-06,  7.50031136e-08,
         -2.46153366e-05, -1.29834876e-17,  2.94396616e-16, -6.32178674e-19,
         -5.26138825e-06, -7.50031136e-08,  2.46214240e-05, -9.88156877e-07,
          7.65319420e-08]),
  'achieved_goal': array([1.20576514, 0.6037528 , 0.42489224]),
  'desired_goal': array([1.29409416, 0.78230083, 0.42489224])},
 {})

In [24]:
import gymnasium as gym
os.path.dirname(gym.__file__), os.path.dirname(os.__file__)


('/Users/christomathew/PycharmProjects/LLM-Robotic-Arm/.venv/lib/python3.12/site-packages/gymnasium',
 '/Users/christomathew/miniforge3/envs/drl_env/lib/python3.12')

In [ ]:
import gymnasium as gym
import numpy as np


class ActiveObjectWrapper(gym.Wrapper):
    """
    Wraps a Fetch-style GoalEnv with multiple objects named object0..object4.

    active_object_index:
        which object index to treat as the task object.
        0 means original object0.
    """

    def __init__(self, env, default_index: int = 0, n_objects: int = 5):
        super().__init__(env)
        self.n_objects = n_objects
        self.active_object_index = default_index

    def set_active_object(self, idx: int):
        """Change which object is used as 'achieved_goal'."""
        if not 0 <= idx < self.n_objects:
            raise ValueError(f"active object index must be in [0, {self.n_objects - 1}]")
        self.active_object_index = idx

    def _get_object_pos(self):
        """Read position of the currently active object from MuJoCo."""
        obj_name = f"object{self.active_object_index}"
        # use site if you made sites 'object0'..'object4'
        pos = self.env.sim.data.get_site_xpos(obj_name).copy()
        # if you only have bodies, use:
        # pos = self.env.sim.data.get_body_xpos(obj_name).copy()
        return pos

    def _remap_obs(self, obs):
        """Replace achieved_goal with selected object's position."""
        pos = self._get_object_pos()
        obs["achieved_goal"] = pos
        return obs

    def reset(self, *, active_object_id=None, **kwargs):
        """
        Optionally pass active_object_id at reset time.
        Objects are already randomized by the env's _reset_sim.
        """
        if active_object_id is not None:
            self.set_active_object(active_object_id)

        obs, info = self.env.reset(**kwargs)
        obs = self._remap_obs(obs)
        return obs, info

    def step(self, action):
        obs, reward, terminated, truncated, info = self.env.step(action)

        # replace achieved_goal with selected object position
        obs = self._remap_obs(obs)

        # recompute reward based on the new achieved_goal
        ag = obs["achieved_goal"]
        dg = obs["desired_goal"]
        # info can be anything, env.compute_reward ignores or uses it
        reward = self.env.compute_reward(ag, dg, info)

        return obs, reward, terminated, truncated, info

In [ ]:
# multi_fetch_env.py

import os
import numpy as np
import gymnasium as gym 
import gymnasium_robotics
from gymnasium_robotics.envs.fetch import MujocoFetchEnv
from gymnasium.utils.ezpickle import EzPickle

GYMROB_FILE = os.path.dirname(gymnasium_robotics.__file__)
MODEL_XML_PATH = os.path.join(
    GYMROB_FILE,
    "envs",
    "assets", 
    "fetch", 
    "multi_pick_and_place.xml"
)


def sample_non_overlapping_xy(
    center_xy, 
    obj_range, 
    n_objects,
    min_dist,
    rng, 
    min_gripper_dist=0.10,
    max_tries = 1000
):
    
    positions = []
    tries = 0
    
    while len(positions)<n_objects and tries < max_tries :
        candidate = center_xy + rng.uniform(-obj_range, obj_range, size=2)
        
        # keep away from gripper
        if np.linalg.norm(candidate - center_xy) < min_gripper_dist:
            tries += 1
            continue
        
        # keep away from other objects
        ok = True
        for p in positions:
            if np.linalg.norm(candidate - p) < min_dist:
                ok = False
                break
                
        
        if ok:
            positions.append(candidate)
        tries += 1
            
    if len(positions) < n_objects:
        raise RuntimeError("Could not sample non-overlapping object positions. Reduce min_dist or increase obj_range.")
    return positions
    
class MultiObjectFetchPickAndPlaceEnv(MujocoFetchEnv, EzPickle):
    def __init__(self, reward_type = 'sparse',n_objects=5,  **kwargs):
        initial_qpos = {
            "robot0:slide0": 0.405,
            "robot0:slide1": 0.48,
            "robot0:slide2": 0.0,
            "object0:joint": [1.25, 0.53, 0.4, 1.0, 0.0, 0.0, 0.0],
            "object1:joint": [1.25, 0.53, 0.4, 1.0, 0.0, 0.0, 0.0],
            "object2:joint": [1.25, 0.53, 0.4, 1.0, 0.0, 0.0, 0.0],
            "object3:joint": [1.25, 0.53, 0.4, 1.0, 0.0, 0.0, 0.0],
            "object4:joint": [1.25, 0.53, 0.4, 1.0, 0.0, 0.0, 0.0],
            
        }
        MujocoFetchEnv.__init__(
                self,
                model_path = MODEL_XML_PATH, 
                has_object=True, 
                block_gripper=False,
                n_substeps=20,
                gripper_extra_height=0.2,
                target_in_the_air=True,
                target_offset=0.0,
                obj_range=0.15,
                target_range=0.15,
                distance_threshold=0.05,
                initial_qpos=initial_qpos,
                reward_type=reward_type,
                **kwargs,
        )
        EzPickle.__init__(self, reward_type=reward_type, n_objects=n_objects, **kwargs)
        self.n_objects = n_objects
        
    def _reset_sim(self):
        # Reset buffers for joint states, actuators, warm-start, control buffers etc.
        self._mujoco.mj_resetData(self.model, self.data)

        self.data.time = self.initial_time
        self.data.qpos[:] = np.copy(self.initial_qpos)
        self.data.qvel[:] = np.copy(self.initial_qvel)
        if self.model.na != 0:
            self.data.act[:] = None
            
        # Randomize start position of object.
        if self.has_object:
            
            center_xy = self.initial_gripper_xpos[:2].copy()
            xy_list = sample_non_overlapping_xy(
                center_xy=center_xy, 
                obj_range=self.obj_range, 
                n_objects=self.n_objects,
                min_dist=self.distance_threshold, 
                rng = self.np_random,
                max_tries=1000
            )
            
            # Table height from object0 default (computed in _env_setup)
            z = float(self.height_offset)
        
            for i, xy in enumerate(xy_list):
                x, y = xy
                joint_name = f"object{i}:joint"
                object_qpos = self._utils.get_joint_qpos(self.model, self.data, joint_name)
                assert object_qpos.shape == (7,)
                object_qpos[0] = x
                object_qpos[1] = y
                object_qpos[2] = z
                # keep orientation as-is (upright), or set quaternion explicitly if you want
                self._utils.set_joint_qpos(
                    self.model, self.data, joint_name, object_qpos 
                )

        # 3) Forward physics
        self._mujoco.mj_forward(self.model, self.data)
        return True
            

In [28]:
# # from multi_fetch_env import MultiObjectFetchPickAndPlaceEnv

# env = MultiObjectFetchPickAndPlaceEnv(render_mode="human")
# obs, info = env.reset()

In [44]:
# env.reset()

({'observation': array([ 1.34193475e+00,  7.49101049e-01,  5.34725189e-01,  1.47273946e+00,
          7.95788880e-01,  4.54922386e-01,  1.30804706e-01,  4.66878310e-02,
         -7.98028028e-02,  3.83587563e-06,  6.44539257e-08,  1.09764540e-02,
         -4.36282865e-02,  1.04072756e-02, -7.31090126e-02, -8.82949285e-05,
         -7.19913982e-02,  5.18577947e-05, -1.25141568e-04,  4.97140072e-05,
         -5.26138825e-06, -7.50031136e-08,  2.46214240e-05, -9.88156877e-07,
          7.65319420e-08]),
  'achieved_goal': array([1.47273946, 0.79578888, 0.45492239]),
  'desired_goal': array([1.49139962, 0.66888359, 0.8966153 ])},
 {})

In [55]:
env.close()

In [68]:
import gymnasium as gym
import numpy as np


class ActiveObjectWrapper(gym.Wrapper):
    """
    Wraps a Fetch-style GoalEnv with multiple objects named object0..object4.

    active_object_index:
        which object index to treat as the task object.
        0 means original object0.
    """

    def __init__(self, env, default_index: int = 0, n_objects: int = 5):
        super().__init__(env)
        self.n_objects = n_objects
        self.active_object_index = default_index

    def set_active_object(self, idx: int):
        """Change which object is used as 'achieved_goal'."""
        if idx<0 or self.n_objects-1<idx:
            raise ValueError(f"Object index must be within the range 0 to {self.n_objects-1}")
        self.active_object_index = idx
        

    def _get_object_pos(self):
        """Read position of the currently active object from MuJoCo."""
        object_name = f"object{self.active_object_index}"
        object_pos = self.env._utils.get_site_xpos( self.env.model, self.env.data, object_name)
        return object_pos
        
    def _remap_obs(self, obs):
        """Replace achieved_goal with selected object's position."""
        obs['achieved_goal'] = self._get_object_pos()
        return obs

    def reset(self, *, active_object_id=None, **kwargs):
        """
        Optionally pass active_object_id at reset time.
        Objects are already randomized by the env's _reset_sim.
        """
        if active_object_id is not None:
            self.set_active_object(active_object_id)
            
        obs, info = self.env.reset(**kwargs)
        obs = self._remap_obs(obs)
        return obs, info
        
    def step(self, action):
        obs, reward, trunc, term, info = self.env.step(action)
        
        obs = self._remap_obs(obs)
        a_g = obs['achieved_goal']
        t_g = obs['desired_goal']
        
        reward = self.env.compute_reward(a_g, t_g, info)
        
        info["is_success"] = self.env._is_success(a_g, t_g)
        
        return obs, reward, trunc, term, info

In [70]:
# from multi_fetch_env import MultiObjectFetchPickAndPlaceEnv

env = MultiObjectFetchPickAndPlaceEnv(render_mode="human")
env = ActiveObjectWrapper(env, 0)
obs, info = env.reset()

In [71]:
obs

{'observation': array([ 1.34193475e+00,  7.49101049e-01,  5.34725189e-01,  1.27766985e+00,
         8.48421594e-01,  4.54922386e-01, -6.42649011e-02,  9.93205454e-02,
        -7.98028028e-02,  3.83587563e-06,  6.44539257e-08,  1.09764540e-02,
        -4.36282865e-02,  1.04072756e-02, -7.31090126e-02, -8.82949285e-05,
        -7.19913982e-02,  5.18577947e-05, -1.25141568e-04,  4.97140072e-05,
        -5.26138825e-06, -7.50031136e-08,  2.46214240e-05, -9.88156877e-07,
         7.65319420e-08]),
 'achieved_goal': array([1.27766985, 0.84842159, 0.45492239]),
 'desired_goal': array([1.25680035, 0.64984908, 0.53460388])}

In [73]:
obs, rew, term, trunc, info = env.step(env.action_space.sample())
obs

{'observation': array([ 1.34221753e+00,  7.32247006e-01,  5.07637729e-01,  1.23434126e+00,
         8.48556203e-01,  4.21328691e-01, -1.07876272e-01,  1.16309198e-01,
        -8.63090380e-02,  2.62543825e-04,  2.79466891e-04,  1.87776364e-03,
        -1.00495698e-02,  1.00625282e-02, -4.62512298e-03,  2.83196542e-03,
         1.58697323e-02, -2.76270440e-03,  1.40567234e-02, -2.51623162e-06,
         4.88676204e-03, -2.77071191e-03, -1.08502708e-02, -2.24994492e-04,
         2.60229594e-04]),
 'achieved_goal': array([1.23434126, 0.8485562 , 0.42132869]),
 'desired_goal': array([1.25680035, 0.64984908, 0.53460388])}

In [77]:
# env.set_active_object(1)
obs, rew, term, trunc, info = env.step(env.action_space.sample())
obs


{'observation': array([ 1.29448436e+00,  7.76726607e-01,  4.49494612e-01,  1.23456787e+00,
         8.48600300e-01,  4.24883280e-01, -5.99164957e-02,  7.18736928e-02,
        -2.46113317e-02,  5.16891807e-02,  5.16287967e-02,  3.66905812e-06,
        -2.05061683e-05,  1.00527260e-02, -8.96794323e-03,  2.15176599e-02,
         1.49021824e-02, -5.67877630e-06,  3.17311585e-05,  7.29709114e-14,
         8.96874831e-03, -2.15175158e-02, -1.48889377e-02, -4.70625186e-03,
        -4.53389253e-03]),
 'achieved_goal': array([1.39842516, 0.86695123, 0.42488275]),
 'desired_goal': array([1.25680035, 0.64984908, 0.53460388])}

In [78]:
env.set_active_object(2)
obs, rew, term, trunc, info = env.step(env.action_space.sample())
obs


{'observation': array([ 1.26827985e+00,  7.45320103e-01,  4.22749553e-01,  1.23456827e+00,
         8.48600372e-01,  4.24890074e-01, -3.37115820e-02,  1.03280268e-01,
         2.14052109e-03,  5.01497040e-02,  5.01458055e-02,  8.41629936e-07,
        -4.70461482e-06,  1.00527260e-02,  2.38741004e-02,  2.21864572e-02,
         1.97117634e-02, -1.27561063e-06,  7.13012399e-06,  1.00521627e-15,
        -2.38739197e-02, -2.21864248e-02, -1.97085829e-02, -3.11979707e-04,
        -3.00315004e-04]),
 'achieved_goal': array([1.47152741, 0.78837976, 0.42488992]),
 'desired_goal': array([1.25680035, 0.64984908, 0.53460388])}

In [ ]:
env.close()

: 

### eval script